In [ ]:
!pip install pandas openpyxl
!pip install tabulate
!pip install linearmodels

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import statsmodels.api as sm
from linearmodels.system import SUR
from statsmodels.tsa.api import VAR

In [ ]:
# Load the Excel file
file_path = "Assignment2Data_2025.xlsx"

# Read a specific sheet
q1 = pd.read_excel(file_path, sheet_name="Q1", engine='openpyxl')

bonus = pd.read_excel(file_path, sheet_name="BonusQuestion", engine='openpyxl')

# Display first few rows
print(q1.head())
print(bonus.head())

# 1

In [ ]:
data_1973_2024 = q1[q1['Calendar Date'] >= 19731231]

# Function to compute statistics
def compute_stats(df):
    stats = {
        "Mean Total Return": round(df["Real R"].mean(),3),
        "Std Total Return": round(df["Real R"].std(),3),
        "Mean Dividend Growth": round(df["Real DG"].mean(),3),
        "Std Dividend Growth": round(df["Real DG"].std(),3),
        "Correlation": round(df[["Real R", "Real DG"]].corr().iloc[0, 1],3)
    }
    return stats

# Compute statistics for both periods
stats_1927_2024 = compute_stats(q1)
stats_1973_2024 = compute_stats(data_1973_2024)

# Create a DataFrame to report results
report = pd.DataFrame([stats_1927_2024, stats_1973_2024], index=["1927-2024", "1973-2024"])
print(report)

In [ ]:
table1 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Mean Total Return", stats_1927_2024['Mean Total Return'], stats_1973_2024['Mean Total Return']],
    ["Std Dev Total Return", stats_1927_2024['Std Total Return'], stats_1973_2024['Std Total Return']],
    ["Mean Dividend Growth", stats_1927_2024['Mean Dividend Growth'], stats_1973_2024['Mean Dividend Growth']],
    ["Std Dev Dividend Growth", stats_1927_2024['Std Dividend Growth'], stats_1973_2024['Std Dividend Growth']],
    ["Correlation (Returns & Growth)", stats_1927_2024['Correlation'], stats_1973_2024['Correlation']],
]

# Print table in Markdown format
print("Table 1. 1927-2024 and 1973-2024 Total Returns, Net Dividend Growth, and Correlations")
print(tabulate(table1, headers="firstrow", tablefmt="pretty"))

# 2

In [ ]:
# 1927-2024
# dividend yield
X = q1['DP'][:-1].reset_index(drop=True)

# real return
Y = q1['Real R'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression, correct for heteroskedasticity
model_1927_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [ ]:
a_1927 = model_1927_2024.params['const']
b_1927 = model_1927_2024.params['DP']
t_stat_a_1927 = model_1927_2024.tvalues['const']
t_stat_b_1927 = model_1927_2024.tvalues['DP']
adj_r2_1927 = model_1927_2024.rsquared_adj

In [ ]:
# 1973-2024
# dividend yield
X = data_1973_2024['DP'][:-1].reset_index(drop=True)

# real return
Y = data_1973_2024['Real R'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1973_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [ ]:
a_1973 = model_1973_2024.params['const']
b_1973 = model_1973_2024.params['DP']
t_stat_a_1973 = model_1973_2024.tvalues['const']
t_stat_b_1973 = model_1973_2024.tvalues['DP']
adj_r2_1973 = model_1973_2024.rsquared_adj

In [ ]:
table2 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Constant", round(a_1927,3), round(a_1973,3)],
    ["Constant t-statistics", round(t_stat_a_1927,3), round(t_stat_a_1973,3)],
    ["Dividend Yield", round(b_1927,3), round(b_1973,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_1927,3), round(t_stat_b_1973,3)],
    ["Adjusted R-Squares", round(adj_r2_1927, 3), round(adj_r2_1973, 3)],
]

# Print table in Markdown format
print("Table 2. Regression Results of 1927-2024 and 1973-2024 Real Returns on Dividend Yield")
print(tabulate(table2, headers="firstrow", tablefmt="pretty"))

# 3

In [ ]:
# 1927-2024
# dividend yield
X = q1['DP'][:-1].reset_index(drop=True)

# dividend growth
Y = q1['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1927_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [ ]:
model_1927_2024.summary()

In [ ]:
a_1927 = model_1927_2024.params['const']
b_1927 = model_1927_2024.params['DP']
t_stat_a_1927 = model_1927_2024.tvalues['const']
t_stat_b_1927 = model_1927_2024.tvalues['DP']
adj_r2_1927 = model_1927_2024.rsquared_adj

In [ ]:
# 1973-2024
# dividend yield
X = data_1973_2024['DP'][:-1].reset_index(drop=True)

# real dividend growth
Y = data_1973_2024['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1973_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [ ]:
a_1973 = model_1973_2024.params['const']
b_1973 = model_1973_2024.params['DP']
t_stat_a_1973 = model_1973_2024.tvalues['const']
t_stat_b_1973 = model_1973_2024.tvalues['DP']
adj_r2_1973 = model_1973_2024.rsquared_adj

In [ ]:
table3 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Constant", round(a_1927,3), round(a_1973,3)],
    ["Constant t-statistics", round(t_stat_a_1927,3), round(t_stat_a_1973,3)],
    ["Dividend Yield", round(b_1927,3), round(b_1973,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_1927,3), round(t_stat_b_1973,3)],
    ["Adjusted R-Squares", round(adj_r2_1927, 3), round(adj_r2_1973, 3)],
]

# Print table in Markdown format
print("Table 3. Regression Results of 1927-2024 and 1973-2024 Real Dividend Growth on Dividend Yield")
print(tabulate(table3, headers="firstrow", tablefmt="pretty"))

# 4

In [ ]:
# 1927-2024
# payout ratio
X = q1['DE'][:-1].reset_index(drop=True)

# dividend growth
Y = q1['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1927_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [ ]:
model_1927_2024.summary()

In [ ]:
a_1927 = model_1927_2024.params['const']
b_1927 = model_1927_2024.params['DE']
t_stat_a_1927 = model_1927_2024.tvalues['const']
t_stat_b_1927 = model_1927_2024.tvalues['DE']
adj_r2_1927 = model_1927_2024.rsquared_adj

In [ ]:
# 1973-2024
# dividend yield
X = data_1973_2024['DE'][:-1].reset_index(drop=True)

# real dividend growth
Y = data_1973_2024['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1973_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [ ]:
a_1973 = model_1973_2024.params['const']
b_1973 = model_1973_2024.params['DE']
t_stat_a_1973 = model_1973_2024.tvalues['const']
t_stat_b_1973 = model_1973_2024.tvalues['DE']
adj_r2_1973 = model_1973_2024.rsquared_adj

In [ ]:
table4 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Constant", round(a_1927,3), round(a_1973,3)],
    ["Constant t-statistics", round(t_stat_a_1927,3), round(t_stat_a_1973,3)],
    ["Dividend Yield", round(b_1927,3), round(b_1973,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_1927,3), round(t_stat_b_1973,3)],
    ["Adjusted R-Squares", round(adj_r2_1927, 3), round(adj_r2_1973, 3)],
]

# Print table in Markdown format
print("Table 4. Regression Results of 1927-2024 and 1973-2024 Real Dividend Growth on Payout Ratio")
print(tabulate(table4, headers="firstrow", tablefmt="pretty"))

Don't predict because R-square is very small. T-statistics are small and are not significant at 5% significance level. 

# 5

In [ ]:
# seemingly unrelated regression

# dividend yield
X = -np.log(q1['DP'][:-1]).reset_index(drop=True)

# dividend growth. Plus 1 because "Real DG" is net growth
Y1 = np.log(q1['Real DG'][1:]+1).reset_index(drop=True)

# dividend yield
Y2 = -np.log(q1['DP'][1:]).reset_index(drop=True)

# drop NA
data = pd.DataFrame({'X': X, 'Y1': Y1, 'Y2': Y2})

# Drop rows with any NA values
data_cleaned = data.dropna()

equations = {
    "eq1": {
        "dependent": data_cleaned["Y1"],
        "exog": sm.add_constant(data_cleaned["X"])
    },
    "eq2": {
        "dependent": data_cleaned["Y2"],
        "exog": sm.add_constant(data_cleaned["X"])
    }
}

# Fit SUR model
sur_model = SUR(equations)
results = sur_model.fit()
results_robust = results.get_robustcov_results(cov_type='HC3')

# Extract covariance matrix of betas
cov_matrix = results_robust.cov

In [ ]:
cov_matrix

In [ ]:
cov_beta = cov_matrix['eq1_X'][3]

In [ ]:
# equation (7)
X = sm.add_constant(data_cleaned['X'])

eq7 = sm.OLS(data_cleaned['Y1'], X).fit(cov_type='HC3')

a_eq7 = eq7.params['const']
b_eq7 = eq7.params['X']
t_stat_a_eq7 = eq7.tvalues['const']
t_stat_b_eq7 = eq7.tvalues['X']
adj_r2_eq7 = eq7.rsquared_adj

In [ ]:
# equation 8
# 1927-2024
eq8 = sm.OLS(data_cleaned['Y2'], X).fit(cov_type='HC3')

a_eq8 = eq8.params['const']
b_eq8 = eq8.params['X']
t_stat_a_eq8 = eq8.tvalues['const']
t_stat_b_eq8 = eq8.tvalues['X']
adj_r2_eq8 = eq8.rsquared_adj

In [ ]:
print(a_eq7)
print(b_eq7)
print(a_eq8)
print(b_eq8)

\begin{align*}
Var(\beta_{pd,r}^{Null})&=Var(-(1-\hat{\beta}_{pd,\Delta d}-\rho\hat{\beta}_{pd,pd+1}))\\
&=Var(1-\hat{\beta}_{pd,\Delta d}-\rho\hat{\beta}_{pd,pd+1})\\
&=Var(1)+Var(\beta_{pd,\Delta d})+\rho^2 Var(\beta_{pd,pd+1})+2\rho Cov(\beta_{pd,\Delta d},\beta_{pd,pd+1})
\end{align*}

In [ ]:
b_se_eq7 = eq7.bse['X']
b_se_eq8 = eq8.bse['X']

In [ ]:
b_var_eq6 = b_se_eq7 ** 2 + (0.94 * b_se_eq8) ** 2 + 2 * 0.94 * cov_beta

In [ ]:
b_eq6 = -(1-b_eq7-0.94*b_eq8)
t_stat_b_eq6 = b_eq6 / (b_var_eq6 ** 0.5)

In [ ]:
diff_eq6 = np.log(q1['Real R'][1:]).reset_index(drop=True) - b_eq6 * -np.log(q1['DP'][:-1]).reset_index(drop=True)
a_eq6 = diff_eq6.mean()

In [ ]:
y = np.log(q1['Real R'][1:]).reset_index(drop=True)

pred = a_eq6 + b_eq6 * -np.log(q1['DP'][:-1]).reset_index(drop=True)
squared_diff = (y - pred) ** 2
sse = squared_diff.sum()

mean = y.mean()

squared_total = (y - mean) ** 2
sst = squared_total.sum()

r2 = 1 - sse/sst

#calculate adjusted r2
n = len(y)
adj_r2_eq6 = 1 - ((1 - r2) * (n - 1)) / (n - 1 - 1)

In [ ]:
table5 = [
    ["Metric", "Return", "Dividend Growth", "Dividend Yield"],
    ["Constant", round(a_eq6,3), round(a_eq7,3), round(a_eq8,3)],
    ["Constant t-statistics", "", round(t_stat_a_eq7,3), round(t_stat_a_eq8,3)],
    ["Dividend Yield", round(b_eq6,3), round(b_eq7,3), round(b_eq8,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_eq6,3), round(t_stat_b_eq7,3), round(t_stat_b_eq8,3)],
    ["Adjusted R-Squares", round(adj_r2_eq6,3), round(adj_r2_eq7,3), round(adj_r2_eq8,3)]
]

# Print table in Markdown format
print("Table 5. Regression Results of Log Real Return, Real Dividend Growth, and Real P/D on Log Real P/D")
print(tabulate(table5, headers="firstrow", tablefmt="pretty"))

# 6

In [ ]:
data = pd.DataFrame({'rt': np.log(q1['Real R']), 'dt': np.log(q1['Real DG']), 'pt-dt': -np.log(q1['DP'])}).dropna()

# Fit the VAR model
model = VAR(data)
var_results = model.fit(maxlags=1)  # Fit the model with lag=1
var_results_robust = results.get_robustcov_results(cov_type='HC3')

# Extract the coefficients and residual covariance matrix
A = var_results_robust.coefs  # Transition matrix Γ
A = np.squeeze(A) # change shape from (1,3,3) to (3,3)
epsilon = var_results_robust.sigma_u

In [ ]:
# variance decomposition

rho = 0.94

eI = np.array([1, 0, 0]) # row vector

#identity Matrix
dim = A.shape[1]
I = np.eye(dim)

# (I - rho * A) ^ -1
x = I - rho * A
inv = np.linalg.inv(x)

# news about future returns
news_r = eI @ (rho * A) @ inv @ epsilon

In [ ]:
# news about future cash flows
news_cf = eI @ (I + rho * A @ inv) @ epsilon